1) Импортируем Библиотеки

In [1]:
import numpy as np #библиотека для работы с массивами
import pandas as pd # библиотека для анализа табличных данных

2) Считываем файлл csv с грибами

In [2]:
df = pd.read_csv('mushrooms.csv')

3) Просматриваем данные

In [3]:
df.head() #вывод первых 5-и строк, что бы убедиться в корректности загрузки

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [4]:
df.shape # возвращает размерность данных (кол-во строк и столбцов)

(8124, 23)

4) Преобразуем категориальные данные в числовые.

Категориальные данные — это данные, качественно характеризующие исследуемый процесс или объект, не имеющие количественного выражения

In [5]:
from sklearn.preprocessing import LabelEncoder # преобразует строковые категории в числа
from sklearn.model_selection import train_test_split # разделяет данные на обучающую и тестовую выборку

In [6]:
le = LabelEncoder() # создает объет для кодирования категориальных признаков

In [7]:
ds = df.apply(func=le.fit_transform) # применяет labelencoder ко всем столбцам преобразую категории в числа

In [8]:
ds.head() # выводит первые 5 строк преобразованных данных

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


5) Подготока данных для модели

In [9]:
data = ds.values
X = data[:, 1:]# х - матрица признаков (все столбцы, кроме первого)
y = data[:, 0]# у - вектор целевой переменной (первый столбец)

In [10]:
X.shape, y.shape # проверяет размерность x и у

((8124, 22), (8124,))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#разделяет на тестовую и обучающую выбору 67/33

# Custom NB

6) Реализация кастомного наивного Байеса

Наивный байесовский классификатор (Naive Bayes classifier) — вероятностный классификатор на основе формулы Байеса со строгим (наивным) предположением о независимости признаков между собой при заданном классе.

Апостериорная вероятность — условная вероятность случайного события при условии того, что известны апостериорные данные, то есть полученные после опыта

In [12]:
#сохраняет обучающие данные внутри класса
class CustomNB:
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    #вычисляет априорную вероятность класса label (доля примеров класса в данных)
    def prior_prob(self, label):
        total = self.y_train.shape[0]
        class_examples = np.sum(self.y_train == label)
        return class_examples / float(total)  # python 2

    # вычисляет условную вероятность P (feature_val | label) для конкретного признака
    def conditional_prob(self, feature_col, feature_val, label):
        X_filtered = self.X_train[self.y_train==label]
        numerator = np.sum(X_filtered[:, feature_col] == feature_val)
        denominator = len(X_filtered)
        return numerator / denominator

    # предсказывает класс для одного примера X_test
    # для каждого класса вычисляет апостериорную вероятность
    # возвращает класс с максимальной вероятностью
    def predict_point(self, X_test):
        classes = np.unique(self.y_train)
        n_features = self.X_train.shape[1]
        post_pro = []
        for label in classes:
            likehood = 1.0
            for feature in range(n_features):
                cond = self.conditional_prob(feature, X_test[feature], label)
                likehood *= cond
            prior = self.prior_prob(label)
            post = prior * likehood
            post_pro.append(post)

        return np.argmax(post_pro)
    # применяет predict_point ко всем примерам в X_test
    def predict(self, X_test):
        result = []
        for point in X_test:
            result.append(self.predict_point(point))
        return np.array(result)
    # вычисляет точность модели (долю правильных предсказаний)
    def score(self, X_test, y_test):
        return (self.predict(X_test) == y_test).mean()

7) Обучение и оценка модели

In [13]:
# создает и обучает модель на тренировочных данных
model = CustomNB()
model.fit(X_train, y_train)

In [14]:
# предсказывает классы для первых 10 примеров тестовой выборки
model.predict(X_test[:10])

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [15]:
# выводит истинные классы для сравнения с предсказаниями
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [16]:
# оценивает точность модели на тестовых данных
model.score(X_test, y_test)

np.float64(0.9973890339425587)